In [1]:
import os
import sys
import logging

import pandas as pd

logger = logging.getLogger("task2_scorer")
LANGUAGES = ('en', 'it', 'fr', 'po', 'ru', 'ge')
LABELS = ('Economic', 'Capacity_and_resources', 'Morality', 'Fairness_and_equality',
          'Legality_Constitutionality_and_jurisprudence', 'Policy_prescription_and_evaluation', 'Crime_and_punishment',
          'Security_and_defense', 'Health_and_safety', 'Quality_of_life', 'Cultural_identity', 'Public_opinion',
          'Political', 'External_regulation_and_reputation')

In [40]:
submission_dir = os.path.join('..', '..', '..', 'predictions', 'evaluations_to_report')

In [41]:
output_dir = './correct_format_submissions'
os.makedirs(output_dir, exist_ok=True)

# Write predictions in expected format

In [43]:
for prediction_filepath in os.listdir(submission_dir):
    language = prediction_filepath.split('_')[0]
    eval_df = pd.read_csv(os.path.join(submission_dir, prediction_filepath), index_col='id')
    submission_df = eval_df.apply(lambda row: ','.join(list(row[row==1].index)), axis=1)
    submission_df.to_csv(os.path.join(output_dir, f'{language}_submission.txt'), sep='\t', header=False)
    print(language)
    print(prediction_filepath)


fr
fr_ComplementNaiveBayes_ROS_y_test.csv
en
en_ComplementNaiveBayes_ROS_y_test.csv
ru
ru_ComplementNaiveBayes_SMOTE_y_test.csv
it
it_ComplementNaiveBayes_ROS_y_test.csv
po
po_LinearSVMDual_y_test.csv
ge
ge_ComplementNaiveBayes_SMOTE_y_test.csv


## Verify their format

Code from the `scorer-subtask-2.py` script

In [73]:
def read_frame_list_from_file(file_full_name):
  """
  Read the list of frame names from a file, one per line.
  """
  with open(file_full_name, encoding='utf-8') as f:
    return [ line.rstrip() for line in f.readlines() ]

def _read_csv_input_file(file_full_name):
  """
  Read a csv file with two columns TAB separated:
   - first column is the id of the example
   - second column is the comma-separated list of labels of the example
  """
  a = {}
  with open(file_full_name, encoding='utf-8') as f:
    for line in f.readlines():
      ind = line.find("\t")
      if ind < 0:
        logger.error('ERROR: the file is supposed to be TAB separated, no TAB found on line' + line)
        sys.exit(1)
      if ind==len(line)-2: # line ends in \t\n
        a[line[0:ind]] = []
      else:
        a[line[0:ind]] = line[ind+1:].rstrip().split(",")
  return a

def _labels_correct(labels, CLASSES, debug=False):
  """
  Make sure all the labels correspond to strings in the CLASSES array
  :param labels: a dictionary with strings as values
  :param CLASSES: a list of allowed labels
  """
  if debug:
    s=""
    for articleid in labels.keys():
      s += ",".join([ l for l in labels[articleid] if l not in CLASSES ])
    return s
  else:
    for articleid in labels.keys():
      for l in labels[articleid]:
        if l not in CLASSES:
          return False
  return True

def _correct_number_of_examples(pred_labels, gold_labels):
  """
  Make sure that the number of predictions is exactly the same as the gold labels
  """
  return len(pred_labels.keys())==len(gold_labels.keys())


def _correct_id_list(pred_labels, gold_labels, debug=False):
  """
  Check that the list of keys of pred_labels is the same as the gold file
  """
  if debug:
    return ", ".join(set(pred_labels.keys()).symmetric_difference(set(gold_labels.keys())))
  return len(set(pred_labels.keys()).symmetric_difference(set(gold_labels.keys())))==0

def correct_format(pred_labels, gold_labels, CLASSES):
  """
  Check whether the format of the prediction file is correct.
  The number of checks that can be performed depends on the availability of the gold labels
  """
  if not _labels_correct(pred_labels, CLASSES):
    logger.error('The following labels in the prediction file are not valid: {}.'
                 .format(_labels_correct(pred_labels, CLASSES, True)))
    return False
  if gold_labels: # we can do further checks if the gold_labels are available
    if not _correct_number_of_examples(pred_labels, gold_labels):
      logger.error('The number of predictions (%d) is not the expected one (%d)'
                   %(len(pred_labels.keys()), len(gold_labels.keys())))
      return False
    if not _correct_id_list(pred_labels, gold_labels):
      logger.error('The list of articles ids is not correct. The following ids are not in the gold file: %s'
                   %(_correct_id_list(pred_labels, gold_labels, True)))
      return False
  return True


In [53]:
frame_file_path = os.path.join('..', '..', '..', 'data', 'scorers' , 'frames_subtask2.txt')
CLASSES = read_frame_list_from_file(frame_file_path)

In [77]:
for submission_file in os.listdir('correct_format_submissions'):
  language = submission_file.split('_')[0]

  pred_labels = _read_csv_input_file(f'correct_format_submissions/{submission_file}')
  gold_labels = os.listdir(os.path.join('..', '..', '..', 'data', 'data', language, 'test-articles-subtask-2'))
  gold_labels = {article_name.split('article')[1].split('.txt')[0]: article_name for article_name in gold_labels}

  if correct_format(pred_labels, gold_labels, CLASSES):
    print(f'Submission file for language: {language} is correct')

  else:
    print(f'ERROR: Submission file for language: {language} is flawed')

Submission file for language: fr is correct
Submission file for language: it is correct
Submission file for language: ge is correct
Submission file for language: po is correct
Submission file for language: en is correct
Submission file for language: ru is correct
